In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
from sklearn import linear_model

In [2]:
!python -m pip install pooch # You might need to run this once  
                             # and then you can delete it..  
                             # Needed for Problem 3.  

## General Sparse Dictionary Learning Problem

So the basic concept we're working with here, summarizing the last couple weeks of lecture is:   

> **Dictionary Learning**: Given data $n\times m$ real-value data matrix $Y$, we want to find a $n\times p$ _dictionary_ $A_{\ast}$ and sparse encoding of $Y$, say $p\times m$ $Z_{\ast}$ which solves
$$
A_{\ast}, Z_{\ast} = \text{arg min}_{A, Z} \left|\left|Y-AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1}, ~ \lambda > 0.
$$
Each column of $A$ is called an _atom_.  We seek _overcomplete_ dictionaries so that $n<p$, and to make the problem more tractable, we should have even more data than we have atoms so that $n<p<m$.  

> **Greedy Algorithms**: To solve 
$$
\text{min}_{A, Z} \left|\left|Y-AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1}
$$
we make an initial guess for $A_{0}$ and then solve in two stages:
\begin{align*}
\text{Find} ~ Z_{1} = & \text{arg min}_{Z} \left|\left|Y-A_{0}Z\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1} \\
\text{Find} ~ A_{1} = & \text{arg min}_{A} \left|\left|Y-AZ_{1}\right|\right|_{F}.      
\end{align*}
Repeat then until you (hopefully) get convergence.  Note, by saying "greedy" we mean that the method will almost certainly converge, and thankfully since this is a convex optimization problem, we are guaranteed that any local minima is a global one.  


> **Computing $D_{A}\left|\left|Y-AZ\right|\right|_{F}^{2}$**: So if we define the function $f(A)=\left|\left|Y-AZ\right|\right|_{F}^{2}$, then we can find its directional derivative at $A$ with respect to the direction $W$, denoted as $<D_{A}f(A), W>=\text{tr}((D_{A}f(A))^{T}W)$, via the formula:
$$
<D_{A}f(A), W> = \lim_{\epsilon\rightarrow 0}\frac{f(A+\epsilon W) - f(A)}{\epsilon} = \left<-2Z(Y-AZ)^{T}, W\right>
$$
Thus, if we want to find $A_{\ast}$ such that $D_{A}f(A)|_{A=A_{\ast}}=0$, then we see that $A_{\ast}$ solves
$$
Z(Y-A_{\ast}Z)^{T} = 0 \implies A_{\ast}ZZ^{T} = YZ^{T}.
$$

> **Moore-Penrose Pseudoinverse**: If for the $p\times m$ real-valued matrix $Z$ we have $\text{rank}(Z)=p$ (remember $p<m$), then we can find the rank-reduced SVD of $Z$ which is $Z = U\Sigma_{p}V_{p}^{T}$ where $\Sigma_{p}$ is $p\times p$ and has strictly positive diagonal entries so $\Sigma_{p}^{-1}$ exists.  Our full rank condition ensures that $U^{T}U=UU^{T}=I$, but we only have $V_{p}^{T}V_{p}=I$ with $V_{p}V_{p}^{T}$ in general being a projection matrix, not the identity. We then see that 
$$
ZZ^{T} = U\Sigma_{p}^{2}U_{p}^{T}
$$
is of full rank and thus invertible.  Therefor the problem
$$
A_{\ast}ZZ^{T} = YZ^{T}
$$
can now be written as 
$$
A_{\ast} = YZ^{-P} = YV_{p}\Sigma_{p}^{-1}U^{T}.
$$

### Example: The Noisy Racoon

So it shouldn't be surprising that dictionary learning is something that you can do via library calls in Scikit-Learn.  To wit, there is a full tutorial that walks you through the process of using dictionary learning to do image denoising at:

[Noisy Raccoon](https://scikit-learn.org/stable/auto_examples/decomposition/plot_image_denoising.html#sphx-glr-auto-examples-decomposition-plot-image-denoising-py)

As you can readily see, you can (and should) download the Jupyter notebook that implements the process soup-to-nuts.  So me making you recreate all of it is... silly.  

That said, I can ask you questions that hopefully make you crawl through the code line-by line, and of course there is always theory to fuss over, so let's get to it.  Note, please make all changes to code in that notebook, but please report results in this one.  Upload both to your homework folder.  

Anyway, in the code, dictionary learning is done via the function `MiniBatchDictionaryLearning()` which solves:

$$
\min_{A, Z} \left|\left|Y - AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1, 1}
$$

where 

$$
\left|\left|Z\right|\right|_{1, 1} = \sum_{j=1}^{p}\sum_{l=1}^{m}|z_{jl}|
$$